## **Solution_Activity 9**


In [ ]:
#@title Import Modules
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import altair as alt
from sklearn.inspection import plot_partial_dependence
from mlxtend.evaluate import feature_importance_permutation


In [ ]:
#@title Load data

df = pd.read_csv('https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter09/Dataset/KDDCup99.csv')
df.head()

In [ ]:
#@title Extracting label and saving in "y"
y = df.pop('label')


In [ ]:
df = pd.get_dummies(df)


In [ ]:
#@title Split the dataset into training and test sets using train_test_split() from sklearn.model_selection.
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=1)

In [ ]:
#@title Create a function that will instantiate and fit RandomForestClassifier using .fit() from sklearn.ensemble
rf_model = RandomForestClassifier(random_state=168)
rf_model.fit(X_train, y_train)

In [ ]:
#@title  Create a function that will predict the outcome for the training and testing sets  using .predict().
train_preds = rf_model.predict(X_train)
test_preds = rf_model.predict(X_test)


In [ ]:
#@title Create a function that will print the accuracy score for the training and testing  sets using accuracy_score() from sklearn.metrics.
train_acc = accuracy_score(y_train, train_preds)
test_acc = accuracy_score(y_test, test_preds)
print(train_acc)
print(test_acc)

In [ ]:
#@title Compute the feature importance via permutation with feature_importance_permutation() and display it on a bar chart using altair
imp_vals, _ = feature_importance_permutation(predict_method=rf_model.predict, X=X_train.values, y=y_train.values, metric='accuracy', num_rounds=1, seed=2)
imp_vals

In [ ]:
perm_varimp_df = pd.DataFrame({'feature': X_train.columns, 'importance': imp_vals})


In [ ]:
perm_varimp_df.sort_values('importance', ascending=False, inplace=True)
perm_varimp_df.head()

In [ ]:
#@title  Plot the partial dependence plot using plot_partial_dependence on the  src_bytes variable.
alt.Chart(perm_varimp_df[:20]).mark_bar().encode(
    x='importance',
    y=alt.Y('feature:N', sort=alt.SortField(field='importance', order='descending'))
)

In [ ]:
#@title Install LIME
!pip install lime

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

In [ ]:

class_names = sorted(y.unique())
class_names

In [ ]:
lime_explainer = LimeTabularExplainer(X_train.values,
      feature_names=X_train.columns,
      class_names=class_names,
      mode='classification')

In [ ]:
#@title Perform a LIME analysis on row 99893 with explain_instance()
exp = lime_explainer.explain_instance(X_test.iloc[99893,], rf_model.predict_proba, num_features=50, top_labels=1)
exp.show_in_notebook()